In [1]:
# load libraries
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [2]:
# Define the transforms 
# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

In [4]:
# Separate the pcitures in different folders for each breed
import os
os.chdir('/home/gabriel/Dropbox/Dataset/dog-breed-identification')

In [5]:
os.listdir('.')

['train_test_validate.zip',
 'train',
 'train_test_validate',
 'test',
 'model_transfer_learning.pth',
 'sample_submission.csv',
 'labels.csv',
 'train_valid']

In [6]:
import pandas as pd
df_labels = pd.read_csv('labels.csv')

In [7]:
df_labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [8]:
# Separate train pictures into breed folders

In [9]:
cd train/

/home/gabriel/Dropbox/Dataset/dog-breed-identification/train


In [11]:
# Iterate over the train folder and copy pictures to respective breed folders
import shutil
for file in os.listdir('.'):
    fname = file.split('.')[0]
    breed = df_labels[df_labels.id == fname]['breed'].values[0]
    if breed not in os.listdir('../organized_train_data'):
        os.mkdir(f'../organized_train_data/{breed}')
    shutil.copy(file, f'../organized_train_data/{breed}/{file}')
    

In [14]:
cd ..

/home/gabriel/Dropbox/Dataset/dog-breed-identification


### Neural Network Trainning Steps

In [7]:
# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder('/home/gabriel/Dropbox/Dataset/dog-breed-identification/organized_train_data', transform=train_transforms)
test_data = datasets.ImageFolder('/home/gabriel/Dropbox/Dataset/dog-breed-identification/small_test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [8]:
# Load our pretrainned model Densenet121
model = models.densenet121(pretrained=True)


In [9]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 120)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

In [17]:
# Load the model from file
#model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load("/home/gabriel/Dropbox/Dataset/dog-breed-identification/model_transfer_learning.pth"))
#model.eval()

#model.load_state_dict(torch.load("/home/gabriel/Dropbox/Dataset/dog-breed-identification/model_transfer_learning.pth"))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [26]:
import time

In [18]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [22]:
"""epochs = 100
steps = 0
running_loss = 0
print_every = 150
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        print(f'Epoch: {epoch}')
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    #print(f'Testloader Inputs: {inputs}')
                    #print(f'Testloader Labels: {labels}')
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()
    torch.save(model.state_dict(), "/home/gabriel/Dropbox/Dataset/dog-breed-identification/model_transfer_learning.pth")
"""

'epochs = 100\nsteps = 0\nrunning_loss = 0\nprint_every = 150\nfor epoch in range(epochs):\n    for inputs, labels in trainloader:\n        steps += 1\n        # Move input and label tensors to the default device\n        inputs, labels = inputs.to(device), labels.to(device)\n        \n        optimizer.zero_grad()\n        \n        logps = model.forward(inputs)\n        loss = criterion(logps, labels)\n        loss.backward()\n        optimizer.step()\n\n        running_loss += loss.item()\n        print(f\'Epoch: {epoch}\')\n        \n        if steps % print_every == 0:\n            test_loss = 0\n            accuracy = 0\n            model.eval()\n            with torch.no_grad():\n                for inputs, labels in testloader:\n                    #print(f\'Testloader Inputs: {inputs}\')\n                    #print(f\'Testloader Labels: {labels}\')\n                    inputs, labels = inputs.to(device), labels.to(device)\n                    logps = model.forward(inputs)\n   

In [21]:
torch.save(model.state_dict(), "/home/gabriel/Dropbox/Dataset/dog-breed-identification/model_transfer_learning.pth")

In [14]:
#model = TheModelClass()
model.load_state_dict(torch.load("/home/gabriel/Dropbox/Dataset/dog-breed-identification/model_transfer_learning.pth"))
model.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [43]:
model.train()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac